# Keras DL Exploration

I am exploring building a deep learning model using the Keras framework in order to solve the Pima Indians Diabetes Database Challenge (https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database). 

This dataset was taken directly from the Kaggle website, which has taken the data from the National Insititute of Diabetes and Digestive and Kidney Diseases. The data was collected for a collection of female patients born of Pima Indian hertiage, aged 21 years or older, in order to predict whether certain physical health measures could act as predictors for whether or not the patient has diabetes.

This notebook is part 1 of 2. In this part (1) I explore building a basic neural network to classify on the dataset with Keras, and in part 2 I experiment with model tuning in order to optimize the performance of the network on the dataset.

In [11]:
#start by importing some of the packages that will be useful
# !pip install tensorflow
from numpy import loadtxt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import pandas as pd

## Load and explore the data

The first 8 features are health metrics for a given individual and the "Outcome" is whether (1) or not (0) the individual has diabetes.

In [2]:
#read in the data file and take a peek
df = pd.read_csv('diabetes.csv')
print(df.shape)
df.head()

(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
#identify the feature matrix and target variable
X = df.drop('Outcome', axis=1)
y = df.Outcome

In [4]:
#perhaps should standardize the features so that no one feature dominates in learning ( some variables are on different scales than others)
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,0.639947,0.848324,0.149641,0.907270,-0.692891,0.204013,0.468492,1.425995
1,-0.844885,-1.123396,-0.160546,0.530902,-0.692891,-0.684422,-0.365061,-0.190672
2,1.233880,1.943724,-0.263941,-1.288212,-0.692891,-1.103255,0.604397,-0.105584
3,-0.844885,-0.998208,-0.160546,0.154533,0.123302,-0.494043,-0.920763,-1.041549
4,-1.141852,0.504055,-1.504687,0.907270,0.765836,1.409746,5.484909,-0.020496


## Define the Keras model

I am going to build up the neural network by sequentially adding layers, each fully connected with a ReLU activation function, and using a sigmoid activation function for the output layer to map predictions to the classification space.

Going to start with two hidden layers, the first with 12 nodes and the second with 8 (the output layer with 1 node, of course); can return and tune the model to optimize for this task afterwards as well.

In [5]:
#define the keras model
model = Sequential()
model.add(Dense(12, input_shape=(8,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

## Compile the Keras model

Going to use binary cross-entropy as the loss function to evaluate the weights in the context of this classification problem, as well as an adam optimizer (advantages of bot AdaGrad and RMSProp optimizers- learning rates tuned based on average first moment as well as the second moments of the gradients) which is both efficient and robust to a wide range of problems.

In [6]:
#compile the model using the loss function and optimizer specified above
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Fit the Keras model

Let's use 150 epochs (number of passes over the entire training dataset) and a batch size (training samples the model uses to learn in a single iteration) of 10 in order to fit the model to the training data. Again, can always come back and tune this at a later point. Let us also use a TensorBoard logging object in order to visualize the training of the model.

In [7]:
#split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) #usign an 80/20 split for now


In [12]:
#create a TensorBoard logger object
logger = TensorBoard(
    log_dir='logs',
    write_graph=True,
    histogram_freq=5
)

In [13]:
#fit the model on the training data (ensure to specify to use the TensorBoard logger object)
model.fit(X_train, y_train, epochs=150, batch_size=10, callbacks=[logger])

Epoch 1/150
62/62 [==============================] - 0s 2ms/step - loss: 0.3362 - accuracy: 0.8371
Epoch 2/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3353 - accuracy: 0.8339
Epoch 3/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3347 - accuracy: 0.8322
Epoch 4/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3334 - accuracy: 0.8322
Epoch 5/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3337 - accuracy: 0.8339
Epoch 6/150
62/62 [==============================] - 0s 2ms/step - loss: 0.3326 - accuracy: 0.8306
Epoch 7/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3330 - accuracy: 0.8322
Epoch 8/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3322 - accuracy: 0.8274
Epoch 9/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3327 - accuracy: 0.8404
Epoch 10/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3338 - accuracy: 0.8290
Epoch 11/

62/62 [==============================] - 0s 1ms/step - loss: 0.3060 - accuracy: 0.8583
Epoch 84/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3079 - accuracy: 0.8583
Epoch 85/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3072 - accuracy: 0.8518
Epoch 86/150
62/62 [==============================] - 0s 2ms/step - loss: 0.3074 - accuracy: 0.8518
Epoch 87/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3060 - accuracy: 0.8550
Epoch 88/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3061 - accuracy: 0.8599
Epoch 89/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3095 - accuracy: 0.8550
Epoch 90/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3061 - accuracy: 0.8664
Epoch 91/150
62/62 [==============================] - 0s 2ms/step - loss: 0.3053 - accuracy: 0.8599
Epoch 92/150
62/62 [==============================] - 0s 1ms/step - loss: 0.3041 - accuracy: 0.8599
Epoch 93/150


## Predict and Evaluate the performance of the Keras model

Evaluate the performance of the model on the testing set.

In [9]:
#make some class predictions
y_pred=(model.predict(X_test)>0.5).astype(int)

#evaluate the model
loss, accuracy=model.evaluate(X_test, y_test)

5/5 [==============================] - 0s 3ms/step - loss: 0.6306 - accuracy: 0.7662


In [ ]:
#lets save the model in case we wish to use it again (without having to train again)
model.save("trained_model_pima1.h5")

## Conclusions: 

The performance of this model is not great on the test set!! I will have to tune the model hyperparameters in order  to improve the model performance!!!